# 37009 Workshop Week 2: Review of Financial Market Instruments (Part 2)

**Note:** This notebook implements the option pricing routines for the workshop exercises. Keep this notebook in hand as you will find useful the creation of functions to implement the Black-Scholes-Merton pricing formulas.

**Reference:** These codes are adapted from Alos, E. & Merino, R. (2023). *Introduction to Financial Derivatives with Python*. CRC Press, Florida, USA.

## European options under the Black-Scholes-Merton model

We first create the functions that provide the prices of European call and put options on an underlying asset that pays a known dividend yield.

In [10]:
# Required modules
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta

# Pricing function for European calls and puts
def BSprice(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option prices
    if option_type == 'call':
        price = (np.exp(-q * tau) * s * sp.stats.norm.cdf(d1, 0.0, 1.0) \
                 - np.exp(-r * tau) * K * sp.stats.norm.cdf(d2, 0.0, 1.0))
        
    if option_type == 'put':
        price = (np.exp(-r * tau) * K * sp.stats.norm.cdf(-d2, 0.0, 1.0) \
                 - np.exp(-q * tau) * s * sp.stats.norm.cdf(-d1, 0.0, 1.0))
        
    return price

In [11]:
# Test option pricing function
call_price = BSprice(100, 105, 0.02, 0.1, 0, 1, 'call')
print(call_price)

put_price = BSprice(100, 105, 0.02, 0.1, 0, 1, 'put')
print(put_price)

3.6895968278176454
8.479806687887631


In [12]:
# Function to calculate option delta under the Black-Scholes-Merton model
def BSdelta(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    
    # Calculate option delta
    if option_type == 'call':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0)
    
    if option_type == 'put':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0) - 1
    
    return value    

In [13]:
# Test option delta function
call_delta = BSdelta(100, 105, 0.02, 0.1, 0, 1, 'call')
print(call_delta)

put_delta = BSdelta(100, 105, 0.02, 0.1, 0, 1, 'put')
print(put_delta)

0.39737567618946446
-0.6026243238105355


In [14]:
# Function to calculate option gamma under the Black-Scholes-Merton model
def BSgamma(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Compute option gamma (same for calls and puts)
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    value = sp.stats.norm.pdf(d1, 0.0, 1.0) / (s * sigma * np.sqrt(tau))
    
    return value

In [15]:
# Test option gamma function
call_gamma = BSgamma(100, 105, 0.02, 0.1, 0, 1)
print(call_gamma)

0.027270898865527086


## European options on indices

This is question (6) in the Workshop Exercises. From the statement of the exercise, we know that $K = 7000$, $s = 6900$, $\sigma = 0.25$, and $q = 0$. We are missing the time to maturity and the relevant risk-free rate.

The time to maturity is the amount of time in years between today, 15 August 2023, and the option maturity date, 1 April 2024. We assume that the risk-free rate is the applicable zero rate interpolated from the given zero rates for the Australian market. 

In [19]:
# Time to maturity
date_today = datetime(2023, 8, 15)
maturity_date = datetime(2024, 4, 1)
time_diff = maturity_date - date_today
time_diff = time_diff.days / 360
print(time_diff)

# Zero rates for domestic and foreign markets
zero_rates = pd.DataFrame({'Tenor':[3/12, 6/12, 9/12],
                          'Domestic':[0.0286, 0.0296, 0.0306],
                          'Foreign':[0.0425, 0.0464, 0.0489]})
print(zero_rates)

# Interpolate required zero rate
risk_free_rate = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Domestic'])
print(risk_free_rate)


0.6388888888888888
   Tenor  Domestic  Foreign
0   0.25    0.0286   0.0425
1   0.50    0.0296   0.0464
2   0.75    0.0306   0.0489
0.030155555555555554


We are now ready to calculate the European put option price, its $\Delta$, and its $\Gamma$ using the functions we have defined above.

In [21]:
# Option price
put_price = BSprice(6900, 7000, time_diff, risk_free_rate, 0.0, 0.25, 'put')

# Option delta
put_delta = BSdelta(6900, 7000, time_diff, risk_free_rate, 0.0, 0.25, 'put')

# Option gamma
put_gamma = BSgamma(6900, 7000, time_diff, risk_free_rate, 0.0, 0.25)

print('The price of the put option per share is AUD', put_price)
print('The delta of the put option per share is AUD', put_delta)
print('The gamma of the put option per share is AUD', put_gamma)

The price of the put option per share is AUD 531.1893737204286
The delta of the put option per share is AUD -0.4505305948330025
The gamma of the put option per share is AUD 0.0002871125249362881


## European currency options

This is question (5) in the Workshop Exercises. We first express the strike price and spot exchange rate as the price in AUD of one KRW.

In [24]:
# Given information
strike_fx = 1 / 875
spot_fx = 1 / 865
sigma = 0.20

We also need to determine the time to maturity and the applicable risk-free rates in the domestic and foreign markets.

In [25]:
# Time to maturity
date_today = datetime(2023, 8, 15)
maturity_date = datetime(2023, 12, 31)
time_diff = maturity_date - date_today
time_diff = time_diff.days / 360

# Interpolate required zero rates
r_dom = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Domestic'])
r_for = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Foreign'])

We can now compute the price of the currency option

In [26]:
# Currency option price. Also calculate delta and gamma
curr_call_price = BSprice(spot_fx, strike_fx, time_diff, r_dom, r_for, sigma, 'call')
curr_call_delta = BSdelta(spot_fx, strike_fx, time_diff, r_dom, r_for, sigma, 'call')
curr_call_gamma = BSgamma(spot_fx, strike_fx, time_diff, r_dom, r_for, sigma)

print('The price of the call option is AUD', curr_call_price, 'per KRW')
print('The delta of the call option is AUD', curr_call_delta, 'per KRW')
print('The gamma of the call option is AUD', curr_call_gamma, 'per KRW')

The price of the call option is AUD 5.916526358285814e-05 per KRW
The delta of the call option is AUD 0.5425742620546902 per KRW
The gamma of the call option is AUD 2770.9291753674343 per KRW
